In [7]:
class MyNaiveBayesClassifier():
    #filename文件名         dataForamt文件格式 attr attr attr attr class 主要是要区分哪一个是类别
    def __init__(self,filename,dataFormat):
        total=0 #类别总数
        #格式为 {i001:次数,"i500":次数} 先验概率和条件概率的计算都离不开它
        classes={} 
        #格式为 {'i001':列:{appearance:次数,both:次数}} 
        counts={}
        #P(i001)=classes.get('i001')/total
        self.prior={}#先验概率
         #用来计算条件概率 P(both|i001)=counts.get(i001).get(列).get(both)/classes.get(i001) 
        self.condition={} #条件概率 
        self.format=dataFormat.strip().split("\t") #解析格式
        
        #读文件
        f=open(filename,'r',encoding="utf8")
        lines=f.readlines()#读取到文件 一行一行的读
        f.close()
        
        #循环每一行数据
        for line in lines:
            total=total+1#统计总数
            fields=line.strip().split('\t') #一行中的每一列
            vector=[]#用来存储每一行中的attr类目下面的特征
            for i in range(len(fields)):
                #判断对应i列的数据类型
                if self.format[i]=='attr':
                    vector.append(fields[i]) #[appearance,sedentary,...]
                if self.format[i]=='class':
                    category=fields[i]#类别条目  i001
                
            classes.setdefault(category,0)#设置成格式 {"i001":次数}
            counts.setdefault(category,{})#设置成格式 {'i001':{},i500:{}}
            #对类别计数 用来计算先验概率
            classes[category]+=1 
            #用于计算条件概率 字典
            col=0 #那一列
            for  columnValue in vector:#循环这一个类的所有特征列
                col+=1
                counts[category].setdefault(col,{})#格式 {'i001':1:{}}
                counts[category][col].setdefault(columnValue,0)#格式 {'i001':1:{appearance:0}}
                #对这一列来进行计算
                counts[category][col][columnValue]+=1
                
            #到这里是将所有的数据循环完之后得到来我想要的数据 接下来就是求先验概率和条件概率
        #多项式方案
        #1.先验概率 {'i001':次数}==>P(i001)
        for (category,num) in classes.items():
            self.prior[category]=num/total
        #2.求出条件概率
        for (category,columns) in counts.items():
            #这里的格式 {i001:col:{}}
            self.condition.setdefault(category,{})
            for (col,valueCount) in columns.items():
                self.condition[category].setdefault(col,{})
                for ( attrValue,count) in valueCount.items():
                    #注意看这里的条件概率的形式
                    self.condition[category][col][attrValue]=count/classes[category]
        
    def classify(self,data):
        """
        后验概率 :对应 k 个类别 ,有多少个后验 概率 然后取最大值
        
        """
        self.result=[]
        for category ,prior in self.prior.items():
            prob=prior
            col=1#第几列
            for colValue in data:#循环这个列的数据
                #根据后验概率的计算公式,只要有一个条件概率不存在 ,则整个公式的值就是0
                if not colValue in self.condition[category][col]:#这一个列中没有出现过这个特征值
                    prob=0
                else:
                    #后验概率=先验概率*条件概率  所有列的验概率乘
                    prob=prob*self.condition[category][col][colValue]
                col=col+1
            self.result.append( (prob,category) )
        return self.result
# =================================    测试==========================    
a=MyNaiveBayesClassifier('i-01',"attr\tattr\tattr\tattr\tclass")
print("先验概率")
print(a.prior)
print("条件概率")
print(a.condition)
result=a.classify(['health','moderate','moderate','yes'])
print("预测结果集")
print(result)

先验概率
{'i100': 0.4, 'i500': 0.6}
条件概率
{'i100': {1: {'both': 0.5, 'appearance': 0.3333333333333333, 'health': 0.16666666666666666}, 2: {'sedentary': 0.5, 'moderate': 0.16666666666666666, 'active': 0.3333333333333333}, 3: {'moderate': 0.8333333333333334, 'aggressive': 0.16666666666666666}, 4: {'yes': 0.3333333333333333, 'no': 0.6666666666666666}}, 'i500': {1: {'health': 0.4444444444444444, 'appearance': 0.3333333333333333, 'both': 0.2222222222222222}, 2: {'sedentary': 0.2222222222222222, 'active': 0.4444444444444444, 'moderate': 0.3333333333333333}, 3: {'moderate': 0.3333333333333333, 'aggressive': 0.6666666666666666}, 4: {'yes': 0.6666666666666666, 'no': 0.3333333333333333}}}
预测结果集
[(0.0030864197530864196, 'i100'), (0.019753086419753083, 'i500')]
